In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 Google Artifact Registry 入门

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/get_started_with_google_artifact_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/get_started_with_google_artifact_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/ml_metadata/get_started_with_google_artifact_registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用谷歌Artifact Registry与Vertex AI。

请查看更多有关[谷歌Artifact Registry文档](https://cloud.google.com/artifact-registry/docs)。

### 目标

在本教程中，您将学习如何使用 'Google Artifact Registry'。

本教程使用以下 Google Cloud ML 服务：

- 'Google Artifact Registry'

执行的步骤包括：

- 创建一个私有的 Docker 仓库。
- 给一个容器镜像打标签，特定于私有 Docker 仓库。
- 将一个容器镜像推送到私有 Docker 仓库。
- 从私有 Docker 仓库中拉取一个容器镜像。
- 删除一个私有的 Docker 仓库。

### 数据集

本教程中未使用任何数据集。对示例数据集的引用仅用于演示目的。

费用

本教程使用谷歌云的可计费组件：

- Vertex AI
- 云存储

了解有关 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing) 的信息，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

## 安装

安装执行笔记本所需的软件包。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform -q

只有Colab：取消下面的注释即可重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在开始之前

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动认证。请按照下面的相关说明进行操作。

1. Vertex AI Workbench
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐户或其他
* 请查看如何将云存储权限授予您的服务帐户，网址为https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建云存储桶

创建存储桶以存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚未存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

设置变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量

谷歌工件库简介

`谷歌工件库`是一个用于存储和管理私有仓库中的工件的服务，包括容器镜像、Helm图表和语言包。它是谷歌云推荐的容器镜像仓库。

了解更多关于[Docker快速入门](https://cloud.google.com/artifact-registry/docs/docker/quickstart)

### 启用Artifact Registry API

首先，您必须为您的项目启用Artifact Registry API服务。

了解更多有关[启用服务](https://cloud.google.com/artifact-registry/docs/enable-service)的信息。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

## 创建私有的Docker仓库

您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

1. 运行`gcloud artifacts repositories create`命令，使用您的区域创建一个新的docker仓库，描述为“docker仓库”。

2. 运行`gcloud artifacts repositories list`命令来验证您的仓库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置身份验证到您的私人存储库

在推送或拉取容器镜像之前，配置Docker以使用`gcloud`命令行工具来对您所在区域的`Artifact Registry`发出的请求进行身份验证。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

获取一个示例容器映像

为了演示目的，您获取（拉取）我们示例容器映像的本地副本：`hello-app:1.0`

In [ ]:
! docker pull us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0

给你的容器镜像打标签

现在你已经有了自己的容器镜像，第一步是给你的镜像打标签。

-使用仓库名称对Docker镜像进行标记，配置docker push命令将镜像推送到特定位置，例如us-central1-docker.pkg.dev。

-`:my-tag`是你要添加到Docker镜像的标签。如果没有指定标签，它默认为`:latest`。

In [ ]:
CONTAINER_NAME = "my-image:my-tag"

! docker tag us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0 us-central1-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{CONTAINER_NAME}

## 将您的镜像推送到您的私人Docker仓库

接下来，将您的容器推送到您的私人Docker仓库。

In [ ]:
! docker push {REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{CONTAINER_NAME}

现在从您的私有Docker仓库中拉取您的镜像，然后再拉取您的容器。

In [ ]:
! docker pull {REGION}-docker.pkg.dev/{PROJECT_ID}/{PRIVATE_REPO}/{CONTAINER_NAME}

删除您的私有 Docker 代码库

最后，当您的私有代码库变得过时时，请使用命令 `gcloud artifacts repositories delete` 来在 `Google Artifact Registry` 中删除它。

In [ ]:
! gcloud artifacts repositories delete {PRIVATE_REPO} --location={REGION} --quiet